# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff3d8d17a60>
├── 'a' --> tensor([[ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770]])
└── 'x' --> <FastTreeValue 0x7ff3d8d11160>
    └── 'c' --> tensor([[-0.9040,  1.0121,  0.3857,  0.2476],
                        [ 1.4982, -0.4532, -1.7826, -0.0595],
                        [-0.5794,  0.6884, -0.9104,  0.3354]])

In [4]:
t.a

tensor([[ 0.7776,  0.5591,  0.5727],
        [ 0.4101, -0.2861, -0.1770]])

In [5]:
%timeit t.a

68.3 ns ± 0.0142 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff3d8d17a60>
├── 'a' --> tensor([[ 0.7294,  0.3860, -0.3280],
│                   [-1.5708, -1.8527,  1.2502]])
└── 'x' --> <FastTreeValue 0x7ff3d8d11160>
    └── 'c' --> tensor([[-0.9040,  1.0121,  0.3857,  0.2476],
                        [ 1.4982, -0.4532, -1.7826, -0.0595],
                        [-0.5794,  0.6884, -0.9104,  0.3354]])

In [7]:
%timeit t.a = new_value

67.9 ns ± 0.0516 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770]]),
    x: Batch(
           c: tensor([[-0.9040,  1.0121,  0.3857,  0.2476],
                      [ 1.4982, -0.4532, -1.7826, -0.0595],
                      [-0.5794,  0.6884, -0.9104,  0.3354]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7776,  0.5591,  0.5727],
        [ 0.4101, -0.2861, -0.1770]])

In [11]:
%timeit b.a

59.7 ns ± 0.0262 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3196,  0.4706, -0.5300],
               [ 0.2355, -0.9630,  0.6525]]),
    x: Batch(
           c: tensor([[-0.9040,  1.0121,  0.3857,  0.2476],
                      [ 1.4982, -0.4532, -1.7826, -0.0595],
                      [-0.5794,  0.6884, -0.9104,  0.3354]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

826 ns ± 3.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 22.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff32400ed00>
├── 'a' --> tensor([[[ 0.7776,  0.5591,  0.5727],
│                    [ 0.4101, -0.2861, -0.1770]],
│           
│                   [[ 0.7776,  0.5591,  0.5727],
│                    [ 0.4101, -0.2861, -0.1770]],
│           
│                   [[ 0.7776,  0.5591,  0.5727],
│                    [ 0.4101, -0.2861, -0.1770]],
│           
│                   [[ 0.7776,  0.5591,  0.5727],
│                    [ 0.4101, -0.2861, -0.1770]],
│           
│                   [[ 0.7776,  0.5591,  0.5727],
│                    [ 0.4101, -0.2861, -0.1770]],
│           
│                   [[ 0.7776,  0.5591,  0.5727],
│                    [ 0.4101, -0.2861, -0.1770]],
│           
│                   [[ 0.7776,  0.5591,  0.5727],
│                    [ 0.4101, -0.2861, -0.1770]],
│           
│                   [[ 0.7776,  0.5591,  0.5727],
│                    [ 0.4101, -0.2861, -0.1770]]])
└── 'x' --> <FastTreeValue 0x7ff32400ecd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 60.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff32ac0bc10>
├── 'a' --> tensor([[ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770],
│                   [ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770],
│                   [ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770],
│                   [ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770],
│                   [ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770],
│                   [ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770],
│                   [ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770],
│                   [ 0.7776,  0.5591,  0.5727],
│                   [ 0.4101, -0.2861, -0.1770]])
└── 'x' --> <FastTreeValue 0x7ff32400eac0>
    └── 'c' --> tensor([[-0.9040,  1.0121,  0.3857,  0.2476],
                        [ 1.4982, -0.4532, -1.7826, -0.0595],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 61.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 66 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.7776,  0.5591,  0.5727],
                [ 0.4101, -0.2861, -0.1770]],
       
               [[ 0.7776,  0.5591,  0.5727],
                [ 0.4101, -0.2861, -0.1770]],
       
               [[ 0.7776,  0.5591,  0.5727],
                [ 0.4101, -0.2861, -0.1770]],
       
               [[ 0.7776,  0.5591,  0.5727],
                [ 0.4101, -0.2861, -0.1770]],
       
               [[ 0.7776,  0.5591,  0.5727],
                [ 0.4101, -0.2861, -0.1770]],
       
               [[ 0.7776,  0.5591,  0.5727],
                [ 0.4101, -0.2861, -0.1770]],
       
               [[ 0.7776,  0.5591,  0.5727],
                [ 0.4101, -0.2861, -0.1770]],
       
               [[ 0.7776,  0.5591,  0.5727],
                [ 0.4101, -0.2861, -0.1770]]]),
    x: Batch(
           c: tensor([[[-0.9040,  1.0121,  0.3857,  0.2476],
                       [ 1.4982, -0.4532, -1.7826, -0.0595],
                       [-0.5794,  0.6884, -0.9104,  0.3354]],
         

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 162 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770],
               [ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770],
               [ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770],
               [ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770],
               [ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770],
               [ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770],
               [ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770],
               [ 0.7776,  0.5591,  0.5727],
               [ 0.4101, -0.2861, -0.1770]]),
    x: Batch(
           c: tensor([[-0.9040,  1.0121,  0.3857,  0.2476],
                      [ 1.4982, -0.4532, -1.7826, -0.0595],
                      [-0.5794,  0.6884, -0.9104,  0.3354],
                      [-0.9040,  1.0121,  0.3857,  0.2476],
                      [ 1.4982, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 259 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

331 µs ± 2.77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
